<a href="https://colab.research.google.com/github/cyFou/testColab/blob/main/testTRM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#source de ce test :
# https://moazharu.medium.com/building-tiny-recursive-model-from-scratch-when-tiny-networks-beat-giants-at-their-own-game-68d9df9e1fdb
# https://github.com/SamsungSAILMontreal/TinyRecursiveModels/blob/main/dataset/build_arc_dataset.py

In [ ]:
!git clone https://github.com/SamsungSAILMontreal/TinyRecursiveModels.git

In [ ]:
!pip install argdantic

In [ ]:
%cd /content/TinyRecursiveModels

# Téléchargement du dataset arc-agi-1 en utilisant le script du dépôt TinyRecursiveModels
!python -m dataset.build_arc_dataset \
  --input-file-prefix kaggle/combined/arc-agi \
  --output-dir data/arc1concept-aug-1000 \
  --subsets training evaluation concept \
  --test-set-name evaluatio

/content/TinyRecursiveModels
[Puzzle 8be77c9e] augmentation not full, only 72
[Puzzle 4258a5f9] augmentation not full, only 576
[Puzzle 3618c87e] augmentation not full, only 575
[Puzzle 2281f1f4] augmentation not full, only 576
[Puzzle 3906de3d] augmentation not full, only 576
[Puzzle aedd82e4] augmentation not full, only 576
[Puzzle 4612dd53] augmentation not full, only 575
[Puzzle 28e73c20] augmentation not full, only 72
[Puzzle f15e1fac] augmentation not full, only 576
[Puzzle 44d8ac46] augmentation not full, only 576
[Puzzle dc433765] augmentation not full, only 575
[Puzzle a5313dff] augmentation not full, only 576
[Puzzle 3f7978a0] augmentation not full, only 576
[Puzzle d4f3cd78] augmentation not full, only 576
[Puzzle 760b3cac] augmentation not full, only 576
[Puzzle cce03e0d] augmentation not full, only 576
[Puzzle ef135b50] augmentation not full, only 576
[Puzzle a3df8b1e] augmentation not full, only 72
[Puzzle 6855a6e4] augmentation not full, only 576
[Puzzle 05f2a901] augmen